In [1]:
import numpy as np
import pandas as pd
from fuzzywuzzy import process

pd.set_option('expand_frame_repr', False)
pd.set_option('display.max_columns', 0)  # Display any number of columns
pd.set_option('display.max_rows', 68)  # Display any number of rows

KENPOM_FILE = 'kenpom.csv'
ESPN_FILE = ('https://projects.fivethirtyeight.com/'
             'march-madness-api/2019/fivethirtyeight_ncaa_forecasts.csv')
EXPORT_FILENAME = 'simple_2019.csv'
FIRST_FOUR_LOSERS = [
    "St. John's (NY)",
    'Temple',
    'North Carolina Central',
    'Prairie View',
]

# Load kenpom data that was previously scraped
kenpom = pd.read_csv(KENPOM_FILE)

# Import 538's info
espn = pd.read_csv(ESPN_FILE)
espn_slice = (espn['gender'] == 'mens') & (espn['forecast_date'] == max(espn['forecast_date']))
team_info = espn[espn_slice][['team_id', 'team_name', 'team_region', 'team_seed']]

In [2]:
# Match Kenpom and ESPN team names
kenpom['team_id'] = np.nan
kenpom['espn_name'] = np.nan
choices = team_info['team_name'].tolist()  # Get all of the ESPN team names
for index, row in kenpom.iterrows():
    team = row['Team']
    match = process.extractOne(team, choices)  # extract the best match for every kenpom name
    if match[1] == 100:  # it's a perfect match
        espn_team_id = team_info[team_info['team_name'] == match[0]]['team_id'].values[0]
        kenpom.loc[index, 'team_id'] = espn_team_id
        choices.remove(match[0])  # remove that team as a possible choice

# Need to do this first, otherwise the >80 match fails since it matches to Iowa St. ¯\_(ツ)_/¯
kenpom.loc[kenpom['Team'] == "St. John's", 'team_id'] = 2599
choices.remove("St. John's (NY)")

scores = {}
for index, row in kenpom[kenpom['team_id'].isna()].iterrows():
    team = row['Team']
    match = process.extractOne(team, choices)
    espn_team_id = team_info[team_info['team_name'] == match[0]]['team_id'].values[0]
    scores[team] = match, espn_team_id
    if match[1] > 80:
        kenpom.loc[index, 'team_id'] = espn_team_id
        choices.remove(match[0])  # remove that team as a possible choice

missing_kenpom = kenpom.loc[kenpom['team_id'].isna(), 'Team'].tolist()
missing_espn = team_info.loc[team_info['team_name'].isin(choices)][['team_name', 'team_id']]

print("Missing Kenpom Teams: {}".format(missing_kenpom))
print("Matching ESPN Teams:\n {}".format(missing_espn))

Missing Kenpom Teams: ['LSU', 'VCU', 'UCF']
Matching ESPN Teams:
                 team_name  team_id
9         Louisiana State       99
16        Central Florida     2116
33  Virginia Commonwealth     2670


In [3]:
scores

{'Michigan St.': (('Michigan State', 88), 127),
 'Florida St.': (('Florida State', 87), 52),
 'Iowa St.': (('Iowa State', 82), 66),
 'LSU': (('Louisiana State', 60), 99),
 'Mississippi St.': (('Mississippi State', 90), 344),
 'Kansas St.': (('Kansas State', 86), 2306),
 "Saint Mary's": (("Saint Mary's (CA)", 95), 2608),
 'Utah St.': (('Utah State', 82), 328),
 'VCU': (('Louisiana State', 30), 99),
 'Ohio St.': (('Ohio State', 82), 194),
 'UCF': (('Louisiana State', 30), 99),
 'New Mexico St.': (('New Mexico State', 90), 166),
 'Murray St.': (('Murray State', 86), 93),
 'Arizona St.': (('Arizona State', 87), 9),
 'Georgia St.': (('Georgia State', 87), 2247),
 'North Dakota St.': (('North Dakota State', 91), 2449),
 'Prairie View A&M': (('Prairie View', 95), 2504)}

In [4]:
# Need to manually fix the crappy ones
kenpom.loc[kenpom['Team'] == 'LSU', 'team_id'] = 99
kenpom.loc[kenpom['Team'] == 'VCU', 'team_id'] = 2670
kenpom.loc[kenpom['Team'] == 'UCF', 'team_id'] = 2116

merged = pd.merge(kenpom, team_info, how='left', on='team_id')

for team in FIRST_FOUR_LOSERS:
    merged = merged[merged['team_name'] != team]

simple = merged[['team_region', 'Seed', 'team_name', 'team_id', 'AdjustEM', 'AdjustT']]

In [5]:
simple.head()

,team_region,Seed,team_name,team_id,AdjustEM,AdjustT
0,South,1,Virginia,258.0,35.66,59.3
1,West,1,Gonzaga,2250.0,32.79,69.8
2,East,1,Duke,150.0,31.99,72.2
3,East,2,Michigan State,127.0,31.36,67.4
4,West,2,Michigan,130.0,29.44,64.4


In [6]:
simple.shape

(64, 6)

In [7]:
simple.to_csv(EXPORT_FILENAME, index=False)